# MCP + LangGraph Hands-On Tutorial

- Author: [Teddy Notes](https://youtube.com/c/teddynote)
- Lecture: [Fastcampus RAG trick notes](https://fastcampus.co.kr/data_online_teddy)

**References**
- https://modelcontextprotocol.io/introduction
- https://github.com/langchain-ai/langchain-mcp-adapters

## configure

Refer to the installation instructions below to install `uv`.

**How to install `uv`**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

Install **dependencies**

```bash
uv pip install -r requirements.txt
```

Gets the environment variables.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## MultiServerMCPClient

Run `mcp_server_remote.py` in advance. Open a terminal with the virtual environment activated and run the server.

> Command
```bash
source .venv/bin/activate
python mcp_server_remote.py
```

Create and terminate a temporary Session connection using `async with`

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

async with MultiServerMCPClient(
    {
        "weather": {
            # Must match the server's port (port 8005)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(
        agent, {"messages": "What's the weather like in Seoul?"}
    )

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x115486520>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I can check the current weather in Seoul for you. Let me get that information right away.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 


You might notice that you can't access the tool because the session is closed.

In [4]:
await astream_graph(agent, {"messages": "What's the weather like in Seoul?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I can check the current weather in Seoul for you. Let me get that information right away.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I apologize for the error. It seems there was an issue with accessing the weather service. Let me try again to get the weather information for Seoul.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I apologize, but it appears there's a technical issue with the weather service at the moment. The system is returning a "ClosedResourceError", which suggests that the weather data service might be temporarily unavailable.

Would you like me to try checking the weather for Seoul again in a moment, or is there so

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-51a733a4-0c98-4cd5-851c-06a2249f51bc', usage_metadata={'input_tokens': 0, 'output_tokens': 77, 'total_tokens': 77, 'input_token_details': {}}),
 'metadata': {'langgraph_step': 5,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:91340b75-51c5-cce9-63c1-3758fbbbff2c',
  'checkpoint_ns': 'agent:91340b75-51c5-cce9-63c1-3758fbbbff2c',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

Now let's change that to accessing the tool while maintaining an Async Session.

In [5]:
# 1. Create client
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. Explicitly initialize connection (this part is necessary)
# Initialize
await client.__aenter__()

# Now tools are loaded
print(client.get_tools())  # Tools are displayed

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11501d580>)]


Create an agent with langgraph(`create_react_agent`).

In [6]:
# Create agent
agent = create_react_agent(model, client.get_tools())

Run the graph to see the results.

In [7]:
await astream_graph(agent, {"messages": "What's the weather like in Seoul?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I can check the current weather in Seoul for you. Let me get that information right away.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in Seoul
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
According to the weather information, it's currently sunny in Seoul. Enjoy the nice weather if you're there or planning to visit!

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-6a076a89-4e53-43b1-96da-b8575ebb66d2', usage_metadata={'input_tokens': 0, 'output_tokens': 31, 'total_tokens': 31, 'input_token_details': {}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:c1a24703-85d1-1c76-cf6a-dbd8decc200b',
  'checkpoint_ns': 'agent:c1a24703-85d1-1c76-cf6a-dbd8decc200b',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## Stdio method

The Stdio method is intended for use in a local environment.

- Use standard input/output for communication

In [8]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_anthropic import ChatAnthropic

# Initialize Anthropic's Claude model
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# Set up StdIO server parameters
# - command: Path to Python interpreter
# - args: MCP server script to execute
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["mcp_server_local.py"],
)

# Use StdIO client to communicate with the server
async with stdio_client(server_params) as (read, write):
    # Create client session
    async with ClientSession(read, write) as session:
        # Initialize connection
        await session.initialize()

        # Load MCP tools
        tools = await load_mcp_tools(session)
        print(tools)

        # Create agent
        agent = create_react_agent(model, tools)

        # Stream agent responses
        await astream_graph(agent, {"messages": "What's the weather like in Seoul?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1218a3e20>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I can check the current weather in Seoul for you. Let me get that information right away.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 


## Use MCP server with RAG deployed

- File: `mcp_server_rag.py`

Use the `mcp_server_rag.py` file that we built with langchain in advance.

It uses stdio communication to get information about the tools, where it gets the `retriever` tool, which is the tool defined in `mcp_server_rag.py`. This file **doesn't** need to be running on the server beforehand.

In [9]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langchain_teddynote.messages import astream_graph

# Initialize Anthropic's Claude model
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# Set up StdIO server parameters for the RAG server
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["./mcp_server_rag.py"],
)

# Use StdIO client to communicate with the RAG server
async with stdio_client(server_params) as (read, write):
    # Create client session
    async with ClientSession(read, write) as session:
        # Initialize connection
        await session.initialize()

        # Load MCP tools (in this case, the retriever tool)
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)

        # Stream agent responses
        await astream_graph(
            agent,
            {
                "messages": "Search for the name of the generative AI developed by Samsung Electronics"
            },
        )


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'll search for information about the generative AI developed by Samsung Electronics.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이

## Use a mix of SSE and Stdio methods

- File: `mcp_server_rag.py` communicates over Stdio
- `langchain-dev-docs` communicates via SSE

Use a mix of SSE and Stdio methods.

In [10]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Initialize Anthropic's Claude model
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# 1. Create multi-server MCP client
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "./.venv/bin/python",
            # Update with the absolute path to mcp_server_rag.py file
            "args": ["./mcp_server_rag.py"],
            # Communicate via stdio (using standard input/output)
            "transport": "stdio",
        },
        "langchain-dev-docs": {
            # Make sure the SSE server is running on port 8765
            "url": "http://teddynote.io:8765/sse",
            # Communicate via SSE (Server-Sent Events)
            "transport": "sse",
        },
    }
)


# 2. Initialize connection explicitly through async context manager
await client.__aenter__()

Error in sse_reader: 


Create an agent using `create_react_agent` in langgraph.

In [13]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in English."
)
agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

Use the `retriever` tool defined in `mcp_server_rag.py` that you built to perform the search.

In [14]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "Use the `retriever` tool to search for the name of the generative AI developed by Samsung Electronics"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I need to clarify something. The tools available to me don't include a `retriever` tool as mentioned in your request. According to my available functions, I only have access to a `get_weather` function that provides weather information for specified locations.

Would you like me to:
1. Explain what I know about Samsung's generative AI without using tools
2. Clarify which tools you'd like me to use
3. Try a different query that works with the available weather tool

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b4e226c4-5967-475d-8158-c4792412eace', usage_metadata={'input_tokens': 0, 'output_tokens': 108, 'total_tokens': 108, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 1,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:606c9180-c533-5fda-d3d9-a22abb26ff8d',
  'checkpoint_ns': 'agent:606c9180-c533-5fda-d3d9-a22abb26ff8d',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

This time, we'll use the `langchain-dev-docs` tool to perform the search.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "Please tell me about the definition of self-rag by referring to the langchain-dev-docs"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I notice that you're asking about self-RAG and want me to use the `langchain-dev-docs` tool, but I don't actually have access to that tool in my current configuration. 

According to my available functions, I only have access to a `get_weather` function that provides weather information for specified locations. I don't have access to a `langchain-dev-docs` tool or any documentation search capabilities at the moment.

Would you like me to:
1. Share what I know about self-RAG based on my general knowledge
2. Try a different query that works with the available weather tool
3. Clarify which tools you'd like me to use

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-a1f2551c-9fd6-4b7f-8eff-ba147a110ebd', usage_metadata={'input_tokens': 0, 'output_tokens': 150, 'total_tokens': 150, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 4,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:62b21008-e602-ebb3-c7ef-7d73d9825743',
  'checkpoint_ns': 'agent:62b21008-e602-ebb3-c7ef-7d73d9825743',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

Use `MemorySaver` to maintain short-term memory, so multi-turn conversations are possible.

In [ ]:
await astream_graph(
    agent,
    {"messages": "Summarize the previous content in bullet points"},
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
# Summary of Previous Interactions

* You asked me to use a `retriever` tool to search for information about Samsung Electronics' generative AI.
* I explained that I don't have access to a `retriever` tool, only a `get_weather` function.
* You then asked me to refer to `langchain-dev-docs` for information about self-RAG.
* I clarified that I don't have access to `langchain-dev-docs` or any documentation search tools.
* The only function available to me is `get_weather`, which provides weather information for specified locations.
* I offered alternative ways to address your questions using my available capabilities.

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-0af9acf3-74d2-4fe2-8f36-1affdc5fc62d', usage_metadata={'input_tokens': 0, 'output_tokens': 149, 'total_tokens': 149, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 7,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:957c519f-9598-bbb0-47e3-2ddc5e4996db',
  'checkpoint_ns': 'agent:957c519f-9598-bbb0-47e3-2ddc5e4996db',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## LangChain-integrated tools + MCP tools

Here we confirm that tools integrated into LangChain can be used in conjunction with existing MCP-only tools.

In [17]:
from langchain_teddynote.tools.tavily import TavilySearch

# Initialize the Tavily search tool (news type, news from the last 3 days)
tavily = TavilySearch(max_results=3, topic="news", days=3)

# Use it together with existing MCP tools
tools = client.get_tools() + [tavily]

Create an agent using `create_react_agent` in langgraph.

In [18]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in English."
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=MemorySaver())

Perform a search using the newly added `tavily` tool.

In [ ]:
await astream_graph(
    agent, {"messages": "Tell me about today's news for me"}, config=config
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'd be happy to help you find today's news. To provide you with current news information, I'll need to use the web search tool.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[{"url": "https://www.avclub.com/kendrick-lama-superbowl-halftime-show-fcc-complaints", "raw_content": "People whined to the FCC about Kendrick Lamar's halftime show\nSkip to the content\nPaste\n| A.V. Club\n| Jezebel\n|*\nSplinter\n×\nLatest\nNews\nFilm\nTV\nMusic\nGames\nAV Undercover\nBooks\nAux\nNewsletter\nInstagram\nTwitter\nYouTube\nFacebook\nPop culture obsessives writing for the pop culture obsessed.\nLatest News TV Film Music Games AV Undercover\nA bunch of people whined to the FCC about Kendrick Lamar's halftime show\n\"The halftime show was terrible with the language and gestures.\"\nBy Emma Keates  |  March 27, 2025 | 9:40am\nPhoto: Jamie Squire/Getty Images\nMusic News Kendrick Lamar\n×*\nCopy Link\n*\nFacebook\n*

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b4ab4c3d-789a-4a30-ab52-5a57b2d314f9', usage_metadata={'input_tokens': 0, 'output_tokens': 308, 'total_tokens': 308, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:5044f4d3-527f-1641-6e8d-6c8f6922493e',
  'checkpoint_ns': 'agent:5044f4d3-527f-1641-6e8d-6c8f6922493e',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

You can see that the `retriever` tool is working smoothly.

In [20]:
await astream_graph(
    agent,
    {
        "messages": "Use the `retriever` tool to search for the name of the generative AI developed by Samsung Electronics"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'd be happy to search for information about the generative AI developed by Samsung Electronics. However, I notice that the tool you mentioned, `retriever`, is not available in my current set of tools. 

The tools I have access to are:
1. `get_weather` - for weather information
2. `tavily_web_search` - for web searches

Let me use the web search tool to find information about Samsung's generative AI:
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[{"url": "https://www.theverge.com/news/639078/samsung-bespoke-ai-jet-ultra-vacuum-text-messages", "raw_content": "Published Time: 2025-03-30T02:00:00+00:00\nSamsung’s latest smart appliances are all about the screens. | The Verge\nSkip to main content\nThe homepage\nSubscribeSign In\n*\nTech\n*\nReviews\n*\nScience\n*\nEntertainment\n*\nAI\n*\nMore\nThe homepage\nMenu\nNavigation Drawer\nLogin\n/\nSign Up\nclose\nSearch\nTech\nAll Tech\nAmazon\nApple\nFace

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-29719d8b-2b0d-403d-a0ae-85cc7af3f217', usage_metadata={'input_tokens': 0, 'output_tokens': 217, 'total_tokens': 217, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 20,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:0b561c2d-fefc-da95-b7d8-d0f3bd64d657',
  'checkpoint_ns': 'agent:0b561c2d-fefc-da95-b7d8-d0f3bd64d657',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## Smithery MCP Server

- Link: https://smithery.ai/

List of tools used:

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - MCP server providing tools for dynamic and reflective problem-solving through structured thinking processes
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - Run terminal commands and manage files with various editing capabilities. Coding, shell and terminal, task automation

**Note**

- When importing tools provided by smithery in JSON format, you must set `"transport": "stdio"` as shown in the example below.

In [22]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Initialize LLM model
model = ChatAnthropic(model="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000)

# 1. Create client
client = MultiServerMCPClient(
    {
        "server-sequential-thinking": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/server-sequential-thinking",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # Add communication using stdio method
        },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # Add communication using stdio method
        },
        "document-retriever": {
            "command": "./.venv/bin/python",
            # Update with the absolute path to the mcp_server_rag.py file
            "args": ["./mcp_server_rag.py"],
            # Communication using stdio (standard input/output)
            "transport": "stdio",
        },
    }
)


# 2. Explicitly initialize connection
await client.__aenter__()

Create an agent using `create_react_agent` in langgraph.

In [23]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig


agent = create_react_agent(model, client.get_tools(), checkpointer=MemorySaver())

`Desktop Commander` 도구를 사용하여 터미널 명령을 실행합니다.

In [24]:
await astream_graph(
    agent,
    {
        "messages": "Draw the folder structure including the current path as a tree. However, exclude the .venv folder from the output."
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'll draw the folder structure as a tree, excluding the .venv folder. Let me get that information for you.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[FILE] .env
[DIR] .git
[FILE] .gitignore
[FILE] .python-version
[DIR] .venv
[FILE] MCP-HandsOn-ENG.ipynb
[FILE] MCP-HandsOn-KOR.ipynb
[FILE] README.md
[FILE] app.py
[FILE] app_KOR.py
[DIR] data
[FILE] mcp_server_local.py
[FILE] mcp_server_rag.py
[FILE] mcp_server_remote.py
[FILE] pyproject.toml
[FILE] requirements.txt
[FILE] uv.lock
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Now I'll check the contents of the data directory:
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[FILE] .gitkeep
[FILE] sample.pdf
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Let me also get the current working directory to show the full path:
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
C

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b6e7108c-a253-4f0a-bd26-dd3ac4bbe933', usage_metadata={'input_tokens': 0, 'output_tokens': 257, 'total_tokens': 257, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 7,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:0bee0d0d-7fe4-691e-bad1-a3dd6b59c20d',
  'checkpoint_ns': 'agent:0bee0d0d-7fe4-691e-bad1-a3dd6b59c20d',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

We'll use the `Sequential Thinking` tool to see if we can accomplish a relatively complex task.

In [ ]:
await astream_graph(
    agent,
    {
        "messages": (
            "Use the `retriever` tool to search for information about generative AI developed by Samsung Electronics, "
            "and then use the `Sequential Thinking` tool to write a report."
        )
    },
    config=config,
)